# **Exploratory Data Analysis**
---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('dark_background')

## **2) Univariate and Multivariate Analysis**
---

### Motivation:

Our primary objective with this exploratory analysis is to identify the features in our data that best represent our understanding of the [root causes of flight delays](../Research/flight_delay_reasons.md) and in so doing, provide the basis for the **accurate** and **reliable** prediction of commercial flight delays. We aim to develop a sense for how well our feature space captures common flight delay reasons in order to determine which features to select, which additional data to gather, and which features should be re-engineered or excluded.

#### [Suggested Considerations](https://github.com/lighthouse-labs/mid-term-project-I/blob/master/exploratory_analysis.ipynb):

1) Test the hypothesis that the delay is from Normal distribution and that mean of the delay is 0. Be careful about the outliers.
2) Is average/median monthly delay different during the year? If so, which months have the biggest delays and what could be the reason?
3) Does the weather affect the delay?
4) How are taxi times changing during the day? Does higher traffic lead to longer taxi times?
5) What is the average percentage of delays that exist prior to departure (*i.e.* are arrival delays caused by departure delays)? Are airlines able to lower the delay during the flights?
6) How many states cover 50% of US air traffic?
7) Test the hypothesis that planes fly faster when there is a departure delay.
8) When (which hour) do most 'LONG', 'SHORT', 'MEDIUM' haul flights take off?
9) Find the top 10 the bussiest airports. Does the greatest number of flights mean that the majority of passengers went through a given airport? How much traffic do these 10 airports cover?
10) Do bigger delays lead to bigger fuel consumption per passenger?